# **Part III - Transactions**

In [6]:
import requests
from pprint import pprint
import pandas as pd
import os
import dask 
from datetime import datetime, timedelta


# **Get data from API**

TO DO : 

- Run la cell en dessous 
- Avant, il faut aller [ici](https://docs.coinapi.io/), rentrer son adresse mail et clicker sur `get a free api key`
- Copier coller la clé dans un fichier dans le directory du notebook : nom du ficher --> `path_file_API`
- Aller dans le fichier `periods_API.txt`
- Choisir l'index (0,7,14,...) --> dans `index`

- Joao : Tu peux faire de 30 à 35 (des fois ça bug, donc va dans periods_API.txt, supprime les colonnes déjà Ok et retente avec une autre clé api).

In [50]:
def divide_time_periods() : 
    # Define the start and end dates
    start_date = datetime(2021, 1, 1)
    end_date = datetime(2021, 9, 1)

    # Generate a list of dates
    date_list = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]
    datetime_strings = [dt.strftime('%Y-%m-%d') for dt in date_list]
    
    #Divide into slots of 7 (we can only make 10 calls per day)
    slot_size = 7
    date_slots = [datetime_strings[i:i+slot_size] for i in range(0, len(datetime_strings), slot_size)]
    
    with open("data/periods_API.txt", "w") as file : 
        for i in range(0, len(datetime_strings), slot_size) :
            data = str(i) + " : " + " ".join(datetime_strings[i:i+slot_size])
            file.write(data + "\n")

In [106]:
# Retrieve API key
def retrieve_api_key(name_file) : 
    api_key = None
    with open(name_file) as file:
        api_key = file.readline().strip()
    return api_key


# Retrieve historical data on Trades
def get_historical_trades(headers, index, day, save_dir) :
    
    parameters = {  
        "symbol_id": "BITSTAMP_SPOT_BTC_USD",
        "time_start" : day+'T00:00',
        "time_end" : day+'T23:59',
        "limit" : 100000, 
    }
    r = requests.get("https://rest.coinapi.io/v1/trades/BITSTAMP_SPOT_BTC_USD/history", 
                     headers=headers, params=parameters)

    # Check if the request was successful (status code 200)
    if r.status_code == 200:
        # Parse and print the trade data
        data = r.json()
        df = pd.DataFrame(data)
        
        #Save as csv 
        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)

        name_file = str(index) + "_" + day
        df.to_parquet(save_dir+name_file, use_deprecated_int96_timestamps=True, compression="brotli")

    else:
        # Print an error message if the request was unsuccessful
        print(f"Error: {r.status_code} - {r.text}")


#To call every day 
def main_get_historical_trades(path_file_API, path_file_periods, index) : 
    api_key = retrieve_api_key(path_file_API)
    headers = {"X-CoinAPI-Key": api_key}

    days_list = None
    #Open the file and get the list of index to scrap (based on index)
    with open(path_file_periods) as file : 
        lines = file.readlines()
        for line in lines : 
            if line.split(":")[0].strip() == str(index) :
                days_list = line.split(":")[1][1:-1].split(" ")
    
    for day in days_list : 
        get_historical_trades(headers, index, day, save_dir = "data/raw/trades/")

#main_get_historical_trades("API_key_affolter.txt", "data/periods_API.txt", 91)
#main_get_historical_trades("API_key_ben.txt", "data/periods_API.txt", 98)
#main_get_historical_trades("API_key_epfl.txt", "data/periods_API.txt", 105)
#main_get_historical_trades("API_key_joanne.txt", "data/periods_API.txt", 112)
#main_get_historical_trades("API_key_reve.txt", "data/periods_API.txt", 119)
#main_get_historical_trades("API_key_data.txt", "data/periods_API.txt", 126)
#main_get_historical_trades("API_key_benProut.txt", "data/periods_API.txt", 133)
#main_get_historical_trades("API_key_anne.txt", "data/periods_API.txt", 140)
#main_get_historical_trades("API_key_benjiplayer.txt", "data/periods_API.txt", 147)
#main_get_historical_trades("API_key_toto.txt", "data/periods_API.txt", 154)
#main_get_historical_trades("API_key_benj.txt", "data/periods_API.txt", 161)
#main_get_historical_trades("API_key_spoteamfy.txt", "data/periods_API.txt", 168)
#main_get_historical_trades("API_key_toto_reve.txt", "data/periods_API.txt", 175)
#main_get_historical_trades("API_key_bene.txt", "data/periods_API.txt", 182)
#main_get_historical_trades("API_key_anne_reve.txt", "data/periods_API.txt", 189)
#main_get_historical_trades("API_key_bene3.txt", "data/periods_API.txt", 196)

In [101]:
pd.read_parquet("data/raw/trades/49_2021-02-24")

,symbol_id,time_exchange,time_coinapi,uuid,price,size,taker_side
0,BITSTAMP_SPOT_BTC_USD,2021-02-24T00:00:35.6550000Z,2021-02-24T00:00:35.7009960Z,5f645096-a79b-439c-8bb8-fdff9d98ea05,48927.33,0.001108,BUY
1,BITSTAMP_SPOT_BTC_USD,2021-02-24T00:00:37.3850000Z,2021-02-24T00:00:37.4532970Z,01120320-0a4d-47a4-907b-97fed023a7b5,48940.36,0.431515,BUY
2,BITSTAMP_SPOT_BTC_USD,2021-02-24T00:00:37.4130000Z,2021-02-24T00:00:37.5146582Z,c4c3f12a-990c-4ceb-b7cd-7945f6479f28,48940.36,0.079944,BUY
3,BITSTAMP_SPOT_BTC_USD,2021-02-24T00:00:37.8320000Z,2021-02-24T00:00:37.8810136Z,65289a89-842e-4d04-bad6-d8d948b5b4be,48940.36,0.284797,BUY
4,BITSTAMP_SPOT_BTC_USD,2021-02-24T00:00:37.8560000Z,2021-02-24T00:00:37.8971214Z,86901c04-39dd-4f31-8ae8-ad1bb5e5a602,48940.36,0.727027,BUY
...,...,...,...,...,...,...,...
53268,BITSTAMP_SPOT_BTC_USD,2021-02-24T23:58:51.3130000Z,2021-02-24T23:58:51.3779515Z,7b883e8e-757b-416c-bfdf-1d372b4a296b,49718.72,0.007998,SELL
53269,BITSTAMP_SPOT_BTC_USD,2021-02-24T23:58:51.3130000Z,2021-02-24T23:58:51.3809663Z,23c7dac2-3e21-4731-a574-c9e04c0817d4,49718.71,0.213482,SELL
53270,BITSTAMP_SPOT_BTC_USD,2021-02-24T23:58:51.4110000Z,2021-02-24T23:58:51.4927866Z,d71c2b19-038c-4b85-adc3-e97c603dc013,49718.71,0.439330,SELL
53271,BITSTAMP_SPOT_BTC_USD,2021-02-24T23:58:57.7960000Z,2021-02-24T23:58:57.8387211Z,934b2c92-d25d-4e55-8991-ee71ae50c907,49705.42,0.006760,SELL


- 08-01
- 11-01
- 29-01
- 